# Bevaringslover 2D og bølgeligningen 

Bølgeligning er vanskeligere enn de andre klassiske andreordens ligninger!

## 1. Hoppebukk (bølgeligning)

[ikke prosjektrelevant]

Kanskje den letteste metoden å implementere på bølgeligning er hoppe-bukk metoden: bølgeligningen

$$
u_{tt} = c^2 u_{xx}
$$

diskretiseres ved å sette (som vanlig) $u^n_m = u(mh, nk)$, og bruke sentraldifferanser på både $u_{tt}$ og $u_{xx}$:

$$
u_{xx}(x,t) = \frac{u(x+h,t)-2u(x,t) +u(x-h,t)}{h^2} + O(h^2)
$$
$$
u_{tt}(x,t) = \frac{u(x,t+k)-2u(x,t) +u(x,t-k)}{k^2} + O(k^2)
$$

Vi ender opp med metoden:

$$
u^{n+1}_m = 2\left(1-\frac{c^2 k^2}{h^2} \right)u^n_m
+ \frac{c^2 k^2}{h^2}(u^n_{m+1} + u^n_{m-1}) - u^n_{m-1}
$$
(Navnet kommer siden valget $\frac{ck}{h}=1$ fører til at leddet $u^n_m$ forsvinner, eller hoppes over).

Eneste utfordring et leddet $u^n_{m-1}$: det betyr at vi må ha både $u(x,0)$ og $u(x,k)$ for å sette i gang metoden. En enkel løsning er å ta i bruk Taylor-utviklingen (siden vi har antakeligvis $u_t(x,0)=g(x)$ som initalbetingelse), dvs

$$
u(x,k) = u(x,0) + k u_t(x,0) + O(k^2) = f(x) + kg(x).
$$



## 2. Bevaringslover 2D

Det er nokså lettvint å oppskalere Lax-Friedrichs metode til den 2D ligningen. Vi får:

$$
\vec{u}^{n+1}_m = \frac{1}{2}(\vec{u}^n_{m+1} + \vec{u}^n_{m-1}) - \frac{k}{2h}\big(\vec{F}(\vec{u}^n_{m+1})-\vec{F}(\vec{u}^n_{m-1})\big)
$$

## Eksempel - Bølgeligning (som en bevaringslov)

Vi minner om at bølgeligningen er en todimensjonal transportligning, som kan sees ved å sette $u=(u_t,u_x)$, da blir bølgeligningen

$$
\vec{u}_t + A \vec{u}_x = 0,\quad 
A=\begin{pmatrix}
0 & -c^2 \\
-1 & 0
\end{pmatrix}
$$

Den kan også skrives som en bevaringslov:

$$
\vec{u}_t + \vec{F}(\vec{u})_x = 0, \quad \vec{F}(u,v)=\begin{pmatrix}
-c^2 v \\
-u
\end{pmatrix}
$$

Vi kan altså løse den med Lax-Friedrichs (se under)

**Oppgave**: hvilke initial-/randebetingelser har vi tatt i bruk?

In [ ]:
import numpy as np

# hastighet i bølgeligning
c = 2

# først komponent av fluksen
def f_u(u,v):
    return -(c**2)*v

# andre komponent av fluksen
def f_v(u,v):
    return -u

# randbetingelsen av fluks f_u på venstre side, lik 0.
def fua(t):
    return 0

# randbetingelsen av fluks f_u på høyre siden, lik 0.
def fub(t):
    return 0

# randbetingelsen av fluks f_v på venstre side, lik 0.
def fva(t):
    return 0

# randbetingelsen av fluks f_v på høyre side, lik 0.
def fvb(t):
    return 0

# todimensjonal Lax-Friedrichs
# utfører et tidssteg, altså returner u_n+1, v_n+1 gitt u_n, v_n
# u0,v0 er verdiene før vi utfører steget, altså henholdsvis u_n og v_n
# trenger fluksen i u- og v- komponentene
# trenger randbetingelse i u- og v-komponentene, ved venstre og høyre, henholdsvis ua, ub, va, vb
# tidssteglengde k, romsteglengde h
# initialtid lik t
def lf2(fluks_u, fluks_v, ua, ub, va, vb, u0, v0, h, k, t):
    n = u0.size
    u1 = np.zeros(n)
    v1 = np.zeros(n)
    for i in np.arange(1,n-1):
        u1[i] = (u0[i-1] + u0[i+1])/2 - (k/h)*(fluks_u(u0[i+1],v0[i+1])-fluks_u(u0[i-1],v0[i-1]))
        v1[i] = (v0[i-1] + v0[i+1])/2 - (k/h)*(fluks_v(u0[i+1],v0[i+1])-fluks_v(u0[i-1],v0[i-1]))
    u1[0] = ua(t)
    u1[-1] = ub(t)
    v1[0] = va(t)
    v1[-1] = vb(t)
    return u1, v1

# sluttid
T = 1
# antall tidssteg
n = 200

# randen på venstre
a = -1
# randen på høyre
b = 1
# antall steg i rom
m = 100

# sett opp n tider t_i mellom 0 og T
t = np.linspace(0,T,n)
# sett opp m punkter x_j mellom a og b
x = np.linspace(a,b,m)

# steglengde tid
k = t[1]-t[0]
# steglengde rom
h = x[1]-x[0]

# setter opp initialverdiene
u = np.zeros((n,m))
v = np.zeros((n,m))

u[0,:] = 0
v[0,:] = np.cos(2*np.pi*x)

# utfører Lax-Friedrichs
for i in np.arange(1,n):
    u[i,:], v[i,:] = lf2(f_u, f_v, fua, fub, fva, fvb, u[i-1,:], v[i-1,:], h,k,t[i])

Vår løsning er egentlig

$$
\vec{u} = \begin{pmatrix}
u_t \\
u_x
\end{pmatrix}
$$

For å få $u$ fra $\vec{u}$ holder det å integrere enten $u_t$ i tid, eller $u_x$ i rom. Vi velger sistnevnte, som gjøres numerisk med en av metodene fra PNS. Vi tar den enkleste

$$
u(x_i,t) = \int_0^{x_i} u_x(x,t) dx \approx \frac{1}{h} \big( u_x(x_0,t) + u_x(x_1,t) + \cdots u_x(x_i, t)\big),
$$

som implementeres i numpy med "cumsum".

In [ ]:
# integrere for å få U fra v
U = np.cumsum(v,axis=1)

import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

line, = ax.plot(x, U[0,:])

def animate(i):
    line.set_ydata(U[i,:])
    return line,

# lag animasjonen
ani = animation.FuncAnimation(fig, animate, interval=20, blit=True, save_count=t.size)

from IPython.display import HTML
HTML(ani.to_jshtml())